In [11]:
from __future__ import print_function
import keras
from keras.datasets import cifar10
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras.layers.noise import GaussianNoise
from keras.layers.noise import GaussianDropout
from keras.layers.normalization import BatchNormalization

import os
import pickle
import numpy as np

In [12]:
batch_size = 1
num_classes = 10
epochs = 1
data_augmentation = True
num_predictions = 20
save_dir = os.path.join(os.getcwd(), 'saved_models')
model_name = 'keras_cifar10_trained_model.h5'

In [14]:
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
completeDatasetX=np.concatenate((x_train,x_test),axis=0)
completeDatasetY=np.concatenate((y_train,y_test),axis=0)
seed = 7
np.random.seed(seed)
a = np.random.randint(0,60000,size=12000)
newXTrain = completeDatasetX[a]
newYTrain=completeDatasetY[a]

print('x_train shape:', newXTrain.shape)
print(newXTrain.shape[0], 'train samples')
#print(x_test.shape[0], 'test samples')

x_train shape: (12000, 32, 32, 3)
12000 train samples


In [15]:
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

In [19]:
model = Sequential()

model.add(Conv2D(32, (3, 3), padding='same',
                 input_shape=x_train.shape[1:]))
model.add(Activation('relu'))
model.add(Conv2D(32, (3, 3)))
model.add(BatchNormalization(mode=0, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GaussianDropout(0.5))
model.add(Dropout(0.25))

model.add(Conv2D(64, (3, 3), padding='same'))
model.add(BatchNormalization(mode=0, axis=1))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(BatchNormalization(mode=0, axis=1))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(GaussianDropout(0.5))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes))
model.add(Activation('softmax'))

/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:7: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:14: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`
/usr/local/src/bluemix_jupyter_bundle.v63/notebook/lib/python2.7/site-packages/ipykernel/__main__.py:17: UserWarning: Update your `BatchNormalization` call to the Keras 2 API: `BatchNormalization(axis=1)`


In [20]:
opt = keras.optimizers.rmsprop(lr=0.0001, decay=1e-6)

In [21]:
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [22]:
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

In [23]:
if not data_augmentation:
    print('Not using data augmentation.')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Using real-time data augmentation.')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        featurewise_center=False,  # set input mean to 0 over the dataset
        samplewise_center=False,  # set each sample mean to 0
        featurewise_std_normalization=False,  # divide inputs by std of the dataset
        samplewise_std_normalization=False,  # divide each input by its std
        zca_whitening=False,  # apply ZCA whitening
        rotation_range=0,  # randomly rotate images in the range (degrees, 0 to 180)
        width_shift_range=0.1,  # randomly shift images horizontally (fraction of total width)
        height_shift_range=0.1,  # randomly shift images vertically (fraction of total height)
        horizontal_flip=True,  # randomly flip images
        vertical_flip=False)  # randomly flip images

    # Compute quantities required for feature-wise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

Using real-time data augmentation.


In [33]:
 model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        steps_per_epoch=x_train.shape[0] // batch_size,
                        epochs=epochs,
                        workers=4)

Epoch 1/1
50000/50000 [==============================] - 806s - loss: 1.8750 - acc: 0.3241   


In [34]:
label_list_path = 'datasets/cifar-10-batches-py/batches.meta'

In [35]:
keras_dir = os.path.expanduser(os.path.join('~', '.keras'))
datadir_base = os.path.expanduser(keras_dir)
if not os.access(datadir_base, os.W_OK):
    datadir_base = os.path.join('/tmp', '.keras')
label_list_path = os.path.join(datadir_base, label_list_path)

In [36]:
with open(label_list_path, mode='rb') as f:
    labels = pickle.load(f)

# Evaluate model with test data set and share sample prediction results
evaluation = model.evaluate_generator(datagen.flow(x_train, y_train,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_train.shape[0] // batch_size,
                                      workers=4)
print('Model Accuracy = %.2f' % (evaluation[1]))

Model Accuracy = 0.20


In [37]:
predict_gen = model.predict_generator(datagen.flow(x_test, y_test,
                                                   batch_size=batch_size,
                                                   shuffle=False),
                                      steps=x_test.shape[0] // batch_size,
                                      workers=4)

In [38]:
for predict_index, predicted_y in enumerate(predict_gen):
    actual_label = labels['label_names'][np.argmax(y_test[predict_index])]
    predicted_label = labels['label_names'][np.argmax(predicted_y)]
    print('Actual Label = %s vs. Predicted Label = %s' % (actual_label,
                                                          predicted_label))
    if predict_index == num_predictions:
        break

Actual Label = cat vs. Predicted Label = automobile
Actual Label = ship vs. Predicted Label = dog
Actual Label = ship vs. Predicted Label = bird
Actual Label = airplane vs. Predicted Label = ship
Actual Label = frog vs. Predicted Label = frog
Actual Label = frog vs. Predicted Label = dog
Actual Label = automobile vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = dog
Actual Label = cat vs. Predicted Label = bird
Actual Label = automobile vs. Predicted Label = truck
Actual Label = airplane vs. Predicted Label = bird
Actual Label = truck vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = dog
Actual Label = truck vs. Predicted Label = dog
Actual Label = ship vs. Predicted Label = truck
Actual Label = dog vs. Predicted Label = dog
Actual Label = horse vs. Predicted Label = bird
Actual Label = ship vs. Predicted Label = truck
Actual Label = frog vs. Predicted Label = dog
Actual Label = horse vs. Predicted La